In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
import re
from pathlib import Path
from functools import partial, cache, cached_property
from itertools import chain
from inspect import signature
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from bhsysapi import BHSysApi

from api.data.base import data_path, radar_path
from api.gui.params import curves, bbg_params, spot_params, fwd_params
from api.instruments.swap import RealSwap, RollingSwap, Fly, Spread
from api.instruments.utils import parse_dates, parse_number, ez_bump
from api.instruments.strategies import FutureStrategy
from api.instruments.signals import MACDSignal, ZScoreThreshold, EnsambleSignal
from api.instruments.cache import data_cache
from api.instruments.future import FutureChain, FutureSpread, FutureFly
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.instruments import config as c
from api.gui.editable_grid import EditableGrid

from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs, SQL, get_price_info
from gioutils.blpw import BlpQuery
from gioutils.gui.pandas import millify
from gioutils.gui.base import View, store, WidgetView, Tabs
from gioutils.utils import (
    RepeatedTimer, 
    parse_swap_tenor_expr, 
    parse_offset,
    today, 
    get_next_n, 
    get_n_from_code,
    date_from_offset, 
    get_bday,
    get_bbg_fut_chain_ticker
)

from techi import api as ti

In [3]:
from api.gui.collectors import get_vol_from_generator, get_rate_from_generator

In [4]:
ccys = [
    'EUR', 
    'USD', 
    'GBP'
]

In [5]:
curves_df = pd.DataFrame(curves).set_index('ccy').loc[ccys]

In [6]:
from gioutils.gui.pandas import millify1

In [7]:
from gioutils.ezutils import LiveCurvesRemote, LiveVolsRemote, LiveInflLocal, get_id, ZCILSHelper, COBInflLocal
lc = LiveCurvesRemote()
lv = LiveVolsRemote(curves=lc)
li = LiveInflLocal()
lci = COBInflLocal()
z = ZCILSHelper()

In [8]:
curves = lc.db_curves
vols = lv.db_curves
infl = li.db_curves

In [9]:
from api.gui.price_grids import PricingGrid

In [143]:
self = PricingGrid()
self

Container(children=[VBox(children=(VBox(children=(Row(children=[Col(children=[HBox(children=(TextField(clearab…

In [141]:
inputs = pd.DataFrame([
    dict(name='f1', instr='irs', generator='USD SOFR', exp='2y', tnr='2y', payout='rec', strike=3, notional=1e6),
    dict(name='f1', instr='irs', generator='USD SOFR', exp='2y', tnr='5y', payout='pay', strike='a',notional=1e6),
    dict(name='f1', instr='irs', generator='USD SOFR', exp='2y', tnr='10y', payout='rec', strike='a',notional=1e6),
    dict(name='f2', instr='irs', generator='USD LIBOR S 3M*', exp='2y', tnr='2y', payout='pay', strike='a',notional=1e6),
    dict(name='f2', instr='irs', generator='USD LIBOR S 3M*', exp='2y', tnr='5y', payout='rec', strike='a',notional=1e6),
    dict(name='f2', instr='irs', generator='USD LIBOR S 3M*', exp='2y', tnr='10y', payout='pay', strike='a', notional=1e6),
    dict(name='sw1', instr='oswp', generator='USD LIBOR S 3M*', tnr='2y', payout='pay', exp='1y', strike='a', notional=1e6),
])

In [142]:
self.price_grid.data = inputs

In [121]:
self.expr_dg

Container(children=[VBox(children=(Row(children=[Col(children=[HBox(children=(Btn(children=[Icon(children=['md…

In [122]:
new_data = self._get_ir_data(
    curves=curves,
    vols=vols,
)
self.dg.data = new_data

In [129]:
r = self.calc_expressions()

In [136]:
os.getlogin()

'gperciaccante'

In [131]:
expr_data = self.expr_dg.dg.data
expr_data = expr_data.loc[expr_data['expression'] != ""]
expr_data = expr_data.dropna(subset='expression')

In [134]:
pd.concat([expr_data[['expression', 'sign', 'target']], r], axis=1)

expression sign  target      live      rate         pv  vol
0         f1    +     NaN  0.097078  0.097078 -4435.6412  NaN
1         f2    +     NaN  0.105501  0.105501        0.0  NaN

In [105]:
g = self.get_groups()

In [59]:

data = self.dg.data
res = []
for index, group in data.groupby('name', sort=False):
    d = dict(
        name=index,
        rate=group['rate'] @ group['wgt'].fillna(1),
        vol=group['vol'] @ group['wgt'].fillna(1),
        pv=group['pv'].sum()
    )
    res.append(d)
res = pd.DataFrame(res)

In [106]:
expr_data = self.expr_dg.dg.data
expr_data = expr_data.loc[expr_data['expression'] != ""]
expr_data = expr_data.dropna(subset='expression')

dg_data = self.get_groups().fillna(0)

In [109]:
self.get_groups()

name      rate       vol            pv     prem     live
0   f1  0.097078       NaN -4.435641e+03      NaN      NaN
1   f2  0.105501       NaN  5.093170e-11      NaN      NaN
2  sw1  0.040616  0.014307  1.033999e+04  0.01034  0.01034

In [108]:
dg_data

name      rate       vol            pv     prem     live
0   f1  0.097078  0.000000 -4.435641e+03  0.00000  0.00000
1   f2  0.105501  0.000000  5.093170e-11  0.00000  0.00000
2  sw1  0.040616  0.014307  1.033999e+04  0.01034  0.01034

In [60]:
if not expr_data.empty and not dg_data.empty:
    cols = [
        'live',
        'vol',
        'pv',
        'prem',
    ]
    res = {}
    for col in cols:
        c = bh.bhTsCalcExpression(
            resultnames=range(len(expr_data)),  # if omitted it doesn't work with multiple expressions at once
            expressions=expr_data['expression'].to_list(),
            varnamesorhndl=dg_data['name'].to_list(),
            varvalues=dg_data[col].to_list()
        )
        c = ensure_series(c)
        res[col] = c

name      rate       vol            pv
0   f1  0.001274       NaN -4.435641e+03
1   f2  0.001243       NaN  5.093170e-11
2  sw1  0.040616  0.014307  1.033999e+04

In [54]:
group['rate'] @ group['wgt']

0.0012741730123537742

In [53]:
group

name instr generator exp  tnr payout strike   notional  wgt      rate  \
key                                                                         
0     f1   irs  USD SOFR  2y   2y    rec      3  1000000.0  1.0  0.032510   
1     f1   irs  USD SOFR  2y   5y    pay      a  1000000.0 -2.0  0.031935   
2     f1   irs  USD SOFR  2y  10y    rec      a  1000000.0  1.0  0.032633   

     vol         pv     ann01  
key                            
0    NaN -4435.6412  1.766865  
1    NaN     0.0000  4.216315  
2    NaN     0.0000  7.799326